<a href="https://colab.research.google.com/github/cookie-pan/Python-Data-Visualization/blob/main/project/project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Project for Week 2 of "Python Data Visualization".
Read World Bank GDP data and create some basic XY plots.
Be sure to read the project description page for further information
about the expected behavior of the program.
@author: Panxin
"""


import csv
import pygal 


def read_csv_as_nested_dict(filename, keyfield, separator, quote):
    """
    Inputs:
      filename  - Name of CSV file
      keyfield  - Field to use as key for rows
      separator - Character that separates fields
      quote     - Character used to optionally quote fields

    Output:
      Returns a dictionary of dictionaries where the outer dictionary
      maps the value in the key_field to the corresponding row in the
      CSV file.  The inner dictionaries map the field names to the
      field values for that row.
    """
    with open (filename,"r") as fp:
      total_data=csv.DictReader(fp,delimiter=separator, quotechar=quote)
      total_dict={}

      for i in total_data:
        single_dict=dict(i)
        total_dict[single_dict.get(keyfield)]=single_dict
        
    return total_dict



def build_plot_values(gdpinfo, gdpdata):
    """
    Inputs:
      gdpinfo - GDP data information dictionary
      gdpdata - A single country's GDP stored in a dictionary whose
                keys are strings indicating a year and whose values
                are strings indicating the country's corresponding GDP
                for that year.
    Output: 
      Returns a list of tuples of the form (year, GDP) for the years
      between "min_year" and "max_year", inclusive, from gdpinfo that
      exist in gdpdata.  The year will be an integer and the GDP will
      be a float.
    """
    min_year=gdpinfo["min_year"]
    max_year=gdpinfo["max_year"]
    gdp_plot=[]
    
    for year in range(min_year,max_year+1):
      if (str(year) in gdpdata.keys()) and len(gdpdata[str(year)])!=0 :
        gdp_plot.append((year,float(gdpdata[str(year)])))

      # else:
      #   print(gdpdata[str(year)]) #没有gdp的年份应该都是空字符串
       
    
    return gdp_plot



def build_plot_dict(gdpinfo, country_list):
    """
    Inputs:
      gdpinfo      - GDP data information dictionary
      country_list - List of strings that are country names

    Output:
      Returns a dictionary whose keys are the country names in
      country_list and whose values are lists of XY plot values 
      computed from the CSV file described by gdpinfo.

      Countries from country_list that do not appear in the
      CSV file should still be in the output dictionary, but
      with an empty XY plot value list.
    """
    plot_dict = {}
    country_info = read_csv_as_nested_dict(gdpinfo.get("gdpfile"), gdpinfo.get("country_name"), 
                                      gdpinfo.get("separator"), gdpinfo.get("quote"))
    
    for country in country_list:
        datas = []
        
        for year in range(gdpinfo.get("min_year"), gdpinfo.get("max_year")+1):
            try:
                #print(year, country_info.get(country).get(str(year)))
                datas.append((int(year), float(country_info.get(country).get(str(year)))))
            except ValueError:
                continue  #空字符串没法转成浮点数，所以不在那个城市key对应value-列表中
            except AttributeError:
                continue
        plot_dict[country] = datas #如果没有这个城市就返回一个空列表
        
    return plot_dict

def render_xy_plot(gdpinfo, country_list, plot_file):
    """
    Inputs:
      gdpinfo      - GDP data information dictionary
      country_list - List of strings that are country names
      plot_file    - String that is the output plot file name
    Output:
      Returns None.
    Action:
      Creates an SVG image of an XY plot for the GDP data
      specified by gdpinfo for the countries in country_list.
      The image will be stored in a file named by plot_file.
    """
    
    gdp_chart = pygal.XY()
    gdp_chart.title = "Plot of GDP for select countries spanning"  + str(gdpinfo.get("min_year")) \
                                        + ' to ' + str(gdpinfo.get("max_year"))
    gdp_chart.y_title = "GDP in current US dollars"
    gdp_chart.x_title = "year"
    for country in country_list:
      gdp_chart.add(country, build_plot_dict(gdpinfo, country_list).get(country))
    gdp_chart.render_to_file(plot_file)

def test_render_xy_plot():
    """
    Code to exercise render_xy_plot and generate plots from
    actual GDP data.
    """
    gdpinfo = {
        "gdpfile": "isp_gdp.csv",
        "separator": ",",
        "quote": '"',
        "min_year": 1960,
        "max_year": 2015,
        "country_name": "Country Name",
        "country_code": "Country Code"
    }

    render_xy_plot(gdpinfo, [], "isp_gdp_xy_none.svg")
    render_xy_plot(gdpinfo, ["China"], "isp_gdp_xy_china.svg")
    render_xy_plot(gdpinfo, ["United Kingdom", "United States"],
                   "isp_gdp_xy_uk+usa.svg")


# Make sure the following call to test_render_xy_plot is commented out
# when submitting to OwlTest/CourseraTest.

test_render_xy_plot()